In [ ]:
import tensorflow as tf
import os
from tensorflow.examples.tutorials.mnist import input_data

def main():
    #==========init==========
    image_size = 28*28
    output_num = 10
    learning_rate = 0.001
    loop_num = 30000
    batch_size = 100

    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

    #==========model==========
    with tf.device("/cpu:9"):
        x = tf.placeholder(tf.float32, [None, image_size])
        W = tf.Variable(tf.zeros([image_size, output_num]))
        b = tf.Variable(tf.zeros([output_num]))
        y = tf.nn.softmax(tf.matmul(x, W)+b)
        y_ = tf.placeholder(tf.float32, [None, output_num])

        cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(corect_prediction, tf.float32))

        init = tf.global_variables_initializer()
        sess = tf.InteractiveSession()
        sess.run(init)
        
        #==========training==========
        for i in range(loop_num):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
            
            if i%1==0:
                print("step", i, "train_accuracy:", sess.run(accuracy, feed_dict = {x: batch_xs, y_: batch_ys}))
        #==========test==========
        print("test_accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images, y_:mnist.test.labels}))

if __name__ == "__main__":
    main()
                                    

上記は [参考サイト](https://miyabi-lab.space/blog/10)のコードだが、エラーが出る。
原因はtensorflow.examples.tutorials.mnist は TensorFlow 2.X 以降では削除されていることである。
代わりに tensorflow_datasets を使うか、keras の datasets を使うかする。

以下、[ChatGPT](https://chatgpt.com/share/67c8f247-c2e4-800e-87ca-6098d749537f) の力を借りて書いた keras パターンを示す。これは Deep Learning from the Basics のもともとのコードに近い。

In [14]:
import tensorflow as tf
from keras.datasets import mnist

# MNISTデータセットのロード
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# データの前処理
x_train, x_test = x_train / 255.0, x_test / 255.0

# モデルの構築
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# モデルのコンパイル
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# モデルの学習
model.fit(x_train, y_train, epochs=5)

# モデルの評価
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc}")


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8824 - loss: 0.4193
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9642 - loss: 0.1208
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9756 - loss: 0.0801
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9824 - loss: 0.0571
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9862 - loss: 0.0444
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9705 - loss: 0.0903
Test Accuracy: 0.9753999710083008


### Result (neurons, accuracy, time)
- 64, 0.9736999869346619, 25s
- 128, 0.9753999710083008, 30s
- 512, 0.9782000184059143, 57s

では、続いて 交差エントロピー誤差（バッチ対応版）を実装してみる。

In [ ]:
import tensorflow as tf
from keras.datasets import mnist
import numpy as np

# MNISTデータセットのロード
(x_train, y_train), (x_test, y_test) = mnist.load_data()

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t]+1e-7)) / batch_size